# Sea ice thickness 
In this notebook, we'll show more detailed analysis and visualization of ICESat2 sea ice thickness measurements and the PIOMAS (Pan-Arctic Ice Ocean Modeling and Assimilation System) modelled sea ice thickness data. We'll also highlight interactive mapping using the hvplot and holoviews package.  

In [ ]:
import xarray as xr # For working with gridded climate data 
import pandas as pd
import numpy as np
import itertools
from utils.read_data_utils import read_book_data # Helper function for reading the data from the bucket
from utils.plotting_utils import compute_gridcell_winter_means, staticArcticMaps, interactiveArcticMaps, interactive_winter_mean_maps, interactive_winter_comparison_lineplot # Plotting

# Plotting dependencies
import cartopy.crs as ccrs
from textwrap import wrap
import hvplot.xarray
import holoviews as hv
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes
GeoAxes._pcolormesh_patched = Axes.pcolormesh # Helps avoid some weird issues with the polar projection 
%config InlineBackend.figure_format = 'retina'
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150 # Sets figure size in the notebook

# Remove warnings to improve display
import warnings 
warnings.filterwarnings('ignore') 

# Read in the data 
We've included a function in the `read_data_utils` module that reads in the book dataset from the public google storage bucketas an xarray.Dataset object.

In [ ]:
print(read_book_data.__doc__) # Print docstring

In [ ]:
book_ds = read_book_data() # Read/download the data 
print(book_ds) # Show dataset

## Restrict data to a given region
We use a region mask of the Arctic to allow us to select for regions of interest, and ignore other regions. This mask is included as a data variable in the ICESat-2 v2 data, and is provided courtesy W. Meier & S. Stewart, National Snow and Ice Data Center (NSIDC). In our analysis, we restrict the data to the Inner Arctic domain, defined as the combined area of the Central Arctic, Beaufort Sea, Chukchi Sea, E Siberian Sea, Laptev Sea and Kara Sea. 

![Arctic region mask](figs/region_mask.png "Inner Arctic region mask")

CA: Central Arctic, BS: Beaufort Sea, CS: Chukchi Sea, ESS: East Siberian Sea, LS: Laptev Sea, KS: Kara Sea, BaS: Barents Sea, EG: East Greenland Sea, GBB: Gulf of Bothnia, Baltic Sea, BB: Baffin Bay & Davis Strait, BeS: Bering Sea, SO: Sea of Okhotsk, GA: Gulf of Alaska. 

Here, we'll restrict the data to the Inner Arctic by selected the key values [1,2,3,4,5,6], corresponding to the six regions that compose the Inner Arctic. 

In [ ]:
innerArctic = [1,2,3,4,5,6] #Inner Arctic
book_ds_all = book_ds.copy()
book_ds = book_ds.where(book_ds.region_mask.isin(innerArctic))

On the map below, you can see how the data we've called the function on has been cut to show only gridcells in the Inner Arctic! 

In [ ]:
date = "Feb 2020"

fig, axes = plt.subplots(1, 2, figsize=(5,5), subplot_kw={'projection':ccrs.NorthPolarStereo(central_longitude=-45)})
im1 = book_ds_all["ice_thickness_int"].sel(time=date)[0].plot.pcolormesh(ax=axes[0], x='longitude', y='latitude', vmin=0, vmax=4, extend='both', transform=ccrs.PlateCarree(), add_colorbar=False)
im2 = book_ds["ice_thickness_int"].sel(time=date)[0].plot.pcolormesh(ax=axes[1], x='longitude', y='latitude', vmin=0, vmax=4, extend='both', transform=ccrs.PlateCarree(), add_colorbar=False)
plt.colorbar(im1, cax=fig.add_axes([0.93, 0.25, 0.025, 0.5]), extend='both', label="sea ice thickness (m)")

axes[0].set_title("\n".join(wrap("Sea ice thickness before restricting, "+date, 32)), fontsize=8)
axes[1].set_title("\n".join(wrap("Inner Arctic sea ice thickness, "+date, 32)), fontsize=8)
axes[0].coastlines()
axes[1].coastlines()

plt.show()

## Set years over which to perform the analysis 
Setting `years = [2018,2019,2020]` indicates that you want to perform the analyses in the notebook for three winter season: winter 2018-19, 2019-20, and 2020-21

In [ ]:
years = [2018,2019,2020]

# ICESat-2 interpolated sea ice thickness maps
Here, we'll overlay sea ice thickness data over a map of the Arctic. To generate the map, we'll use interactive plotting functions based off the `hvplot` python package. To see more information on the functions used, click on the plus buttons to see the function documentation, or go into the `plotting_utils` module to see the code. 

## Monthly data 
Use the slider to scroll through each month with available data

In [ ]:
print(interactiveArcticMaps.__doc__) # Print docstring 

In [ ]:
pl = interactiveArcticMaps(book_ds["ice_thickness_int"], frame_width=500, vmin=0, vmax=4)
display(pl)

## Gridcell winter means 
The plotting function below builds on the plotting function used above, `interactiveArcticMaps`, to display gridcell winter mean ice thickness. You can toggle through the winter seasons using the dropdown on the right of the map.

In [ ]:
print(interactive_winter_mean_maps.__doc__) # Print docstring

In [ ]:
pl = interactive_winter_mean_maps(book_ds["ice_thickness_int"], 
                                  years=years,  
                                  slider=True, 
                                  frame_width=500, 
                                  title="ICESat-2 sea ice thickness winter means")
display(pl)

# Mean monthly sea ice thickness 
Compare the monthly means across different winter seasons for the uninterpolated ICESat-2 sea ice thickness data

In [ ]:
print(interactive_winter_comparison_lineplot.__doc__)

In [ ]:
pl = interactive_winter_comparison_lineplot(book_ds.ice_thickness, 
                            years=[2018,2019,2020],
                            title="ICESat-2 sea ice thickness winter comparison", 
                            frame_width=500, frame_height=250)
display(pl)